Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [24]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [25]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [26]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [27]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [28]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  weights_h0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h0 = tf.Variable(tf.zeros([num_hidden_nodes]))
  h0 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h0) + biases_h0)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(h0, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights_h0) + tf.nn.l2_loss(biases_h0) + tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.25).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction_output = tf.nn.softmax(logits)


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction_output], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction_output.eval(
                {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(prediction_output.eval(
            {tf_train_dataset : test_dataset, tf_train_labels : test_labels}), test_labels))

Initialized
Minibatch loss at step 0: 570.143372
Minibatch accuracy: 9.4%
Validation accuracy: 24.2%
Minibatch loss at step 500: 161.701416
Minibatch accuracy: 82.8%
Validation accuracy: 62.2%
Minibatch loss at step 1000: 127.603317
Minibatch accuracy: 83.6%
Validation accuracy: 64.6%
Minibatch loss at step 1500: 110.575958
Minibatch accuracy: 83.6%
Validation accuracy: 65.0%
Minibatch loss at step 2000: 95.852554
Minibatch accuracy: 80.5%
Validation accuracy: 64.9%
Minibatch loss at step 2500: 85.521713
Minibatch accuracy: 82.8%
Validation accuracy: 64.5%
Minibatch loss at step 3000: 74.042038
Minibatch accuracy: 87.5%
Validation accuracy: 65.7%
Test accuracy: 90.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [29]:
small_train_dataset = train_dataset[0:512, :]
small_train_labels = train_labels[0:512, :]
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction_output], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction_output.eval(
                {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(prediction_output.eval(
            {tf_train_dataset : test_dataset, tf_train_labels : test_labels}), test_labels))

Initialized
Minibatch loss at step 0: 433.424133
Minibatch accuracy: 6.2%
Validation accuracy: 22.7%
Minibatch loss at step 500: 138.649689
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Minibatch loss at step 1000: 122.357300
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Minibatch loss at step 1500: 107.978653
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Minibatch loss at step 2000: 95.289650
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Minibatch loss at step 2500: 84.092209
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Minibatch loss at step 3000: 74.210510
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Test accuracy: 80.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [30]:
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  weights_h0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h0 = tf.Variable(tf.zeros([num_hidden_nodes]))
  # Add a dropout.
  keep_prob = tf.placeholder(tf.float32)
  h0 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_h0) + biases_h0), keep_prob)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(h0, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights_h0) + tf.nn.l2_loss(biases_h0) + tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.25).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction_output = tf.nn.softmax(logits)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, prediction_output], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction_output.eval(
                {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels, keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(prediction_output.eval(
            {tf_train_dataset : test_dataset, tf_train_labels : test_labels,  keep_prob: 1.0}), test_labels))


Initialized
Minibatch loss at step 0: 641.008545
Minibatch accuracy: 8.6%
Validation accuracy: 26.0%
Minibatch loss at step 500: 139.475861
Minibatch accuracy: 98.4%
Validation accuracy: 60.0%
Minibatch loss at step 1000: 123.300484
Minibatch accuracy: 99.2%
Validation accuracy: 59.8%
Minibatch loss at step 1500: 108.068901
Minibatch accuracy: 100.0%
Validation accuracy: 60.9%
Minibatch loss at step 2000: 95.377007
Minibatch accuracy: 100.0%
Validation accuracy: 61.0%
Minibatch loss at step 2500: 84.172958
Minibatch accuracy: 100.0%
Validation accuracy: 60.8%
Minibatch loss at step 3000: 74.286041
Minibatch accuracy: 100.0%
Validation accuracy: 61.1%
Test accuracy: 84.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [61]:
batch_size = 256
num_hidden_nodes = 1024
num_hidden_nodes1 = 512
num_hidden_nodes2 = 128
num_steps = 20001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  weights_h0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes], stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases_h0 = tf.Variable(tf.zeros([num_hidden_nodes]))
  # Add a dropout.
  keep_prob = tf.placeholder(tf.float32)
  h0 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_h0) + biases_h0), keep_prob)

  weights_h1 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_hidden_nodes1], stddev=np.sqrt(2.0 / num_hidden_nodes)))
  biases_h1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(h0, weights_h1) + biases_h1), keep_prob)

  weights_h2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes)))
  biases_h2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2), keep_prob)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(h2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights_h0) + tf.nn.l2_loss(biases_h0) + tf.nn.l2_loss(weights_h1) + tf.nn.l2_loss(biases_h1) + tf.nn.l2_loss(weights_h2) + tf.nn.l2_loss(biases_h2) + tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss += 1e-4 * regularizers
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  prediction_output = tf.nn.softmax(logits)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, prediction_output], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction_output.eval(
                {tf_train_dataset : valid_dataset, keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(prediction_output.eval(
            {tf_train_dataset : test_dataset,  keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.456793
Minibatch accuracy: 10.2%
Validation accuracy: 16.6%
Minibatch loss at step 500: 0.644806
Minibatch accuracy: 84.4%
Validation accuracy: 70.7%
Minibatch loss at step 1000: 0.558094
Minibatch accuracy: 87.1%
Validation accuracy: 72.8%
Minibatch loss at step 1500: 0.469625
Minibatch accuracy: 90.2%
Validation accuracy: 74.3%
Minibatch loss at step 2000: 0.619600
Minibatch accuracy: 86.7%
Validation accuracy: 75.1%
Minibatch loss at step 2500: 0.396416
Minibatch accuracy: 91.8%
Validation accuracy: 75.3%
Minibatch loss at step 3000: 0.501305
Minibatch accuracy: 87.9%
Validation accuracy: 75.0%
Minibatch loss at step 3500: 0.511345
Minibatch accuracy: 88.7%
Validation accuracy: 76.0%
Minibatch loss at step 4000: 0.417341
Minibatch accuracy: 91.0%
Validation accuracy: 76.2%
Minibatch loss at step 4500: 0.493590
Minibatch accuracy: 88.7%
Validation accuracy: 76.5%
Minibatch loss at step 5000: 0.456712
Minibatch accuracy: 89.8%
Validation accurac